In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/.gitattributes
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00002-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/gener

In [2]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/2", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/kaggle/input/llama-3.1/transformers/8b-instruct/2 does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # 4x longer contexts auto supported!
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
from datasets import load_dataset

# Load CSV files correctly
dataset_train = load_dataset('csv', data_files="/kaggle/input/multi-lingual-sentiment-analysis/train.csv",split='train')
dataset_test = load_dataset('csv', data_files="/kaggle/input/multi-lingual-sentiment-analysis/test.csv")

# Ensure formatting_prompts_func is defined before calling map
# dataset_train=dataset_train
# dataset_train[0]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
# Define Alpaca-style prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Perform sentiment analysis on the following sentence, positive or negative.

### Input:
{}

### Response:
{}"""

# Ensure EOS token is present
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    texts = []
    for sentence, label in zip(examples["sentence"], examples["label"]):
        text = alpaca_prompt.format(sentence, label) + EOS_TOKEN
        texts.append(text)

    # Tokenize with padding & truncation enabled
    tokenized_inputs = tokenizer(
        texts,
        padding="max_length",  # Ensures batch compatibility
        truncation=True,  
        max_length=max_seq_length,
        return_tensors="np",  # Convert to NumPy to avoid tensor issues
    )

    return {
        "input_ids": tokenized_inputs["input_ids"].tolist(),
        "attention_mask": tokenized_inputs["attention_mask"].tolist(),
        "texts":texts
    }

# Apply tokenization with batched processing
dataset = dataset_train.map(formatting_prompts_func, batched=True, remove_columns=dataset_train.column_names)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
from trl import SFTTrainer

from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "texts",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)


Applying chat template to train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 20
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.381600
2,1.527700
3,1.444100
4,1.389400
5,1.678900
6,1.140700
7,1.126400
8,1.587200
9,1.224300
10,1.139800


In [10]:
test_df = pd.read_csv('/kaggle/input/multi-lingual-sentiment-analysis/test.csv')
test_df

,ID,sentence,language
0,1,"1120 mAh, ਓਵਰਚਾਰਜਿੰਗ ਦੀ ਸੁਰੱਖਿਆ",pa
1,2,તે સઘન મોઇશ્ચરાઇઝિંગ પ્રદાન કરે છે અને સરસ સ્વ...,gu
2,3,"1120 ಎಂಎಎಚ್, ಮಿತಿಮೀರಿದ ರಕ್ಷಣೆ",kn
3,4,ভাৰতত নিৰ্মিত সৰ্বশ্ৰেষ্ঠ পাৰফিউম ব্ৰেণ্ডবোৰৰ ...,as
4,5,"میں نے حال ہی میں ""انفولڈ"" سے ایک ٹیمپلیٹ خرید...",ur
...,...,...,...
95,96,"దర్శకత్వం చాలా బాగుంది, మరియు నటన కూడా బాగుంది...",te
96,97,ଏହା ଭଲ କ୍ୱାଲିଟିର ଗଦି ସହିତ ଭଲ ପ୍ୟାଡେଡ୍ ହୋଇଛି |,or
97,98,"അടുത്തിടെ ഞാൻ ""Unfold"" എന്നതിൽ നിന്ന് ഒരു ടെംപ...",ml
98,99,"""സംസ്ഥാനത്ത് നല്ല ബന്ധമാണുള്ളത്. നിങ്ങൾക്ക് സം...",ml


In [11]:
import pandas as pd

FastLanguageModel.for_inference(model)  # Ensure model is in inference mode

predictions = []  # Store extracted predictions

for i in test_df['sentence']:
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                f"Provide sentiment analysis of {i}",  # Instruction
                "",  # Output - left blank for generation
            )
        ],
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to("cuda")

    # Generate prediction
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=200,  # Adjust max tokens
        use_cache=True
    )

    # Decode output
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Extract the sentiment prediction
    if "### Response:" in generated_text:
        prediction = generated_text.split("### Response:")[-1].strip()
    else:
        prediction = generated_text.strip()  # Fallback if formatting differs

    # Remove unwanted tokens (like <|eot_id|>)
    prediction = prediction.replace("<|eot_id|>", "").strip()

    # Append to list
    predictions.append(prediction)



In [12]:
# Create a submission DataFrame with ID and Prediction
submission_df = pd.DataFrame({
    "ID": test_df.index+1,  # Assuming IDs are the index
    "label": predictions  # Extracted sentiment
})

# Save to CSV
submission_df.to_csv("submission.csv", index=False)

print("Submission file saved as submission.csv")


Submission file saved as submission.csv


In [13]:
submission_df

,ID,label
0,1,Negative
1,2,Positive
2,3,Positive
3,4,Positive
4,5,Negative
...,...,...
95,96,Positive
96,97,Positive
97,98,Negative
98,99,Positive


In [13]:
# FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!

# for i in test_df['sentence']:
#     inputs = tokenizer(
#     [
#         alpaca_prompt.format(
#             f"Provide sentiment analysis of {i}", # instruction
#             # f"{i}", # input
#             "", # output - leave this blank for generation!
#         )
#     ], return_tensors = "pt").to("cuda")
    
#     outputs = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
#                              max_new_tokens = 64, use_cache = True)
#     print(tokenizer.batch_decode(outputs))